## Sales Cleaning Template

At my last internship, another intern was doing some data cleaning to turn the raw sales data he was working with into a different form so he could make his data report. I asked if I could take a look and he showed me the data and explained the objective: there were separate tables for categories like sales, sales target, budget, etc., and in each two-way table the columns were dates and the rows were markets. He needed to combine all these separate tables into one table that had the columns Market, Date, Sales, Sales Target, etc. The way he was going about this was by doing Vlookups a couple rows at a time, one column at a time. It was going to take him probably a week. I made him this reusable Jupyter Notebook template in an hour that outputs the desired data in seconds, and with a little walkthrough he could use it on other datasets that he needed to processed in the same fashion. So, although the purpose of the code is rather simple, it's incredible practical, user-friendly, and automates a previously time-consuming and tedious process.

In [1]:
# Import packages:
import pandas as pd
import numpy as np
import math
import warnings

warnings.filterwarnings('ignore')

#### Replace file name if needed:

In [2]:
# Import data
sales_data = pd.read_csv("sales_raw_data.csv")

The data in its original format below shows multiple two-way tables stacked one after the other. The data should not contain separate names for each pivot table. The markets and dates are the same for each table.

Disclaimer: I've replaced the numbers in the original dataset with random numbers for privacy purposes.

#### Review data to see that it looks correct:

In [3]:
sales_data

,Total Sales,Oct FY15,Nov FY15,Dec FY15,Jan FY15,Feb FY15,Mar FY15,Apr FY15,May FY15,Jun FY15,...,Jul-FY20,Aug-FY20,Sept-FY20,Oct-FY21,Nov-FY21,Dec-FY21,Jan-FY21,Feb-FY21,Mar-FY21,Apr-FY21
0,Carmel,10,20,30,40,50,60,70,80,90,...,700,710,720,730,740,750,760,770,780,790
1,Fishers,20,30,40,50,60,70,80,90,100,...,710,720,730,740,750,760,770,780,790,800
2,Greenwood,30,40,50,60,70,80,90,100,110,...,720,730,740,750,760,770,780,790,800,810
3,Noblesville,40,50,60,70,80,90,100,110,120,...,730,740,750,760,770,780,790,800,810,820
4,North Indianapolis,50,60,70,80,90,100,110,120,130,...,740,750,760,770,780,790,800,810,820,830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,Detroit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-491%,1890%,-4448%,5092%,21159%,2392%,6285%
1256,East Lansing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-498%,1913%,-4506%,5155%,21426%,2421%,6364%
1257,Grand Rapids,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-505%,1936%,-4564%,5218%,21693%,2450%,6443%
1258,Southfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-512%,1959%,-4622%,5281%,21960%,2479%,6522%


#### Replace 'Total Sales' below with what's in the A1 cell of your Excel file, then check to see that only market names remain below:

In [4]:
a1 = 'Total Sales'

# Output unique items in first column
markets = sales_data[a1].unique()
markets

array(['Carmel', 'Fishers', 'Greenwood', 'Noblesville',
       'North Indianapolis', 'South Indianapolis', 'Zionsville',
       'Batavia', 'Dekalb', 'East Moline', 'Geneva', 'Genoa',
       'North Aurora', 'Oswego', 'Plainfield', 'Plano', 'Rock Island',
       'Romeoville', 'Sandwich', 'South Elgin', 'St. Charles',
       'Sugar Grove', 'Sycamore', 'Yorkville', 'Berea', 'Lexington',
       'Nicholasville', 'Richmond', 'Versailles', 'Lansing', 'Ypsilanti',
       'Austin', 'Cologne', 'Jaguar', 'Lakeville', 'Mancato',
       'Northfield', 'Rochester', 'Victoria', 'Waconia', 'Watertown',
       'Ames', 'Bettendorf', 'Clive', 'Davenport', 'Grimes', 'Johnston',
       'Le Claire', 'Nevada', 'Urbandale', 'Fayetteville', 'Greenville',
       'Hope Mills', 'Raeford', 'Spring Lake', 'Clayton', 'Englewood',
       'Huber Heights', 'Tipp City', 'Troy', 'Union', 'Vandalia',
       'West Milton', 'Tallahassee', 'Connersville', 'Crawfordsville',
       'Franklin', 'Greencastle', 'Huntington', 'Lafay

In [5]:
# Find number of markets and number of montsh
num_markets = len(markets)
num_months = sales_data.shape[1] - 1

#### Copy-paste the array of market names outputted above to replace the list below:

In [6]:
# Make new dataframe with market names
new_table = pd.DataFrame(columns = ['Market'])
new_table['Market'] = ['Carmel', 'Fishers', 'Greenwood', 'Noblesville',
       'North Indianapolis', 'South Indianapolis', 'Zionsville',
       'Batavia', 'Dekalb', 'East Moline', 'Geneva', 'Genoa',
       'North Aurora', 'Oswego', 'Plainfield', 'Plano', 'Rock Island',
       'Romeoville', 'Sandwich', 'South Elgin', 'St. Charles',
       'Sugar Grove', 'Sycamore', 'Yorkville', 'Berea', 'Lexington',
       'Nicholasville', 'Richmond', 'Versailles', 'Lansing', 'Ypsilanti',
       'Austin', 'Cologne', 'Jaguar', 'Lakeville', 'Mancato',
       'Northfield', 'Rochester', 'Victoria', 'Waconia', 'Watertown',
       'Ames', 'Bettendorf', 'Clive', 'Davenport', 'Grimes', 'Johnston',
       'Le Claire', 'Nevada', 'Urbandale', 'Fayetteville', 'Greenville',
       'Hope Mills', 'Raeford', 'Spring Lake', 'Clayton', 'Englewood',
       'Huber Heights', 'Tipp City', 'Troy', 'Union', 'Vandalia',
       'West Milton', 'Tallahassee', 'Connersville', 'Crawfordsville',
       'Franklin', 'Greencastle', 'Huntington', 'Lafayette', 'Lebanon',
       'Madison', 'New Castle', 'Seymour', 'Vincennes', 'Wabash',
       'Westfield', 'BloomingtonNormal', 'Ann Arbor', 'Detroit',
       'East Lansing', 'Grand Rapids', 'Southfield', 'Toledo']

In [7]:
# Duplicate each market name n times, where n is the number of months, then sort alphabetically. 
final_table = pd.concat([new_table]*num_months, ignore_index=True).sort_values('Market')

#### Check to see that dates below are accurate:

In [8]:
dates = pd.DataFrame({'date': ['Oct FY15', 'Nov FY15', 'Dec FY15', 'Jan FY15', 'Feb FY15', 'Mar FY15',
       'Apr FY15', 'May FY15', 'Jun FY15', 'Jul FY15', 'Aug FY15', 'Sept FY15',
       'Oct FY16', 'Nov FY16', 'Dec FY16', 'Jan FY16', 'Feb FY16', 'Mar FY16',
       'Apr FY16', 'May FY16', 'Jun FY16', 'Jul FY16', 'Aug FY16', 'Sept FY16',
       'Oct FY17', 'Nov FY17', 'Dec FY17', 'Jan FY17', 'Feb FY17', 'Mar FY17',
       'Apr FY17', 'May FY17', 'Jun FY17', 'Jul FY17', 'Aug FY17', 'Sept FY17',
       'Oct-FY18', 'Nov-FY18', 'Dec-FY18', 'Jan-FY18', 'Feb-FY18', 'Mar-FY18',
       'Apr-FY18', 'May-FY18', 'Jun-FY18', 'Jul-FY18', 'Aug-FY18', 'Sept-FY18',
       'Oct-FY19', 'Nov-FY19', 'Dec-FY19', 'Jan-FY19', 'Feb-FY19', 'Mar-FY19',
       'Apr-FY19', 'May-FY19', 'Jun-FY19', 'Jul-FY19', 'Aug-FY19', 'Sept-FY19',
       'Oct-FY20', 'Nov-FY20', 'Dec-FY20', 'Jan-FY20', 'Feb-FY20', 'Mar-FY20',
       'Apr-FY20', 'May-FY20', 'Jun-FY20', 'Jul-FY20', 'Aug-FY20', 'Sept-FY20',
       'Oct-FY21', 'Nov-FY21', 'Dec-FY21', 'Jan-FY21', 'Feb-FY21', 'Mar-FY21',
       'Apr-FY21']})

# Fill in the dates for each market, which becomes the second column of the final table.
dates_expanded = pd.concat([dates]*num_markets, ignore_index=True)
final_table['Month-FY'] = dates_expanded['date'].values

In [9]:
# Function that sorts pivot table by market and converts table values into a list
def pivot_to_array(n):
    return sales_data.iloc[(n)*num_markets : (n+1)*num_markets].sort_values(a1).drop(columns = [a1]).values.flatten()

#### Replace with the column names you want in your final table:

In [10]:
# Make a list of the remaining columns for the final table
column_names = ['Sales', 'SSU', 'RSA', 'DSA & Reseller', 'Close Ratio', 'Sales Budget', 'Sales to Budget', 'Sales Target', 
                'Sales to Target', 'OE Call Sales', 'Net New Subs', 'Net New Budget', 'Net New to Budget', 'Net New Target', 
                'Net New to Target']

# Store the number of columns and a list of indexes for the columns
num_columns = len(column_names)
col_numbers = list(range(num_columns))

In [11]:
# Convert each of the separate pivot tables into lists
matrix = list(map(pivot_to_array, col_numbers))

In [12]:
# Assign each list to a column. The values line up to the specific market and date because of how everything is sorted.
for i in range(num_columns):
    final_table[column_names[i]] = matrix[i]

In [13]:
# Output final table
final_table

,Market,Month-FY,Sales,SSU,RSA,DSA & Reseller,Close Ratio,Sales Budget,Sales to Budget,Sales Target,Sales to Target,OE Call Sales,Net New Subs,Net New Budget,Net New to Budget,Net New Target,Net New to Target
3317,Ames,Oct FY15,420,1260,2100,2940,42%,79,41%,NaN,NaN,42,126,210,NaN,NaN,NaN
3485,Ames,Nov FY15,430,1270,2110,2950,43%,80,42%,NaN,NaN,43,127,211,NaN,NaN,NaN
3569,Ames,Dec FY15,440,1280,2120,2960,44%,81,43%,NaN,NaN,44,128,212,NaN,NaN,NaN
3653,Ames,Jan FY15,450,1290,2130,2970,45%,82,44%,NaN,NaN,45,129,213,NaN,NaN,NaN
3737,Ames,Feb FY15,460,1300,2140,2980,46%,83,45%,NaN,NaN,46,130,214,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4794,Zionsville,Dec-FY21,810,1650,2490,3330,81%,83,80%,-357,-208%,81,165,249,42964%,-941,-214%
4038,Zionsville,Jan-FY21,820,1660,2500,3340,82%,84,81%,-58,1089%,82,166,250,43532%,-467,493%
1602,Zionsville,Feb-FY21,830,1670,2510,3350,83%,85,82%,135,528%,83,167,251,44100%,-331,1668%
6306,Zionsville,Mar-FY21,840,1680,2520,3360,84%,86,83%,-91,269%,84,168,252,44668%,-341,275%


#### Uncomment code below and run notebook:

In [14]:
#final_table.to_csv("sales_cleaned.csv")